## Load Mojo Kernels

In [20]:
import os, numpy, torch
from pathlib import Path
from max.torch import CustomOpLibrary

assert torch.cuda.is_available()
op_dir = os.path.abspath('operations')

## Simple `add_one` Operation

In [32]:
op_lib = CustomOpLibrary(Path(op_dir))
add_one = op_lib.my_add_constant[{"value": 1}]

In [12]:
import time

def torch_add_one(inputs):
    return inputs + 1

def mojo_add_one(inputs):
    outputs = torch.zeros_like(inputs)
    add_one(outputs, inputs)
    return outputs

for device in ["cpu", "cuda"]:
    for op in [torch_add_one, mojo_add_one]:
        x = torch.zeros(1024, device=device)
        x = op(x) # warm-up
        start = time.perf_counter()
        for _ in range(1000):
            x = op(x)
        end = time.perf_counter()
        print(op.__name__, device, x, end - start)

torch_add_one cpu tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.]) 0.0034156449837610126
mojo_add_one cpu tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.]) 0.13251558190677315
torch_add_one cuda tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.], device='cuda:0') 0.0074024859350174665
mojo_add_one cuda tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.], device='cuda:0') 0.11501298600342125


## Different MatMul Algorithms

In [21]:
from max.driver import Accelerator, accelerator_count, Tensor
import torch
M, K, N = 8, 8, 16 # debug
M, K, N = 4096, 6144, 2048 # real-world scale
device = Accelerator()
torch_A = torch.randn(M, K)
torch_B = torch.randn(K, N)
torch_result = (torch_A @ torch_B).numpy()
A = Tensor.from_numpy(torch_A.numpy()).to(device)
B = Tensor.from_numpy(torch_B.numpy()).to(device)

Build and test executing the CUDA graph for our MatMul kernel:

In [22]:
from max.graph import Graph, TensorType, DeviceRef, ops
def build_graph(session, algorithm):
    print('building cuda graph for', algorithm)
    with Graph("matmul_graph",
               input_types=[
                   TensorType(dtype=A.dtype, shape=A.shape, device=DeviceRef.from_device(device)),
                   TensorType(dtype=B.dtype, shape=B.shape, device=DeviceRef.from_device(device))
               ],
               custom_extensions=[Path(op_dir)]) as graph:
        A_value, B_value = graph.inputs
        output = ops.custom(
            name="my_matmul",
            device=DeviceRef.from_device(device),
            values=[A_value, B_value],
            out_types=[
                TensorType(dtype=A.dtype, shape=[
                        A_value.tensor.shape[0], B_value.tensor.shape[1]
                    ], device=DeviceRef.from_device(device))
            ],
            parameters={"algorithm": algorithm},
        )
        graph.output(output[0].tensor)
    print('loading cuda graph...')
    return session.load(graph) # compile the graph

from max.engine import InferenceSession
session = InferenceSession(devices=[device])
graph =  build_graph(session, "block_tiling")
mojo_result = graph.execute(A, B)[0].to_numpy()
print("test run:\n", mojo_result, end="\n\n")
print("reference:\n", torch_result)

building cuda graph for block_tiling
loading cuda graph...
test run:
 [[ -55.741146   -13.114622    27.26144   ...   30.763079   168.56628
    60.865597 ]
 [ 151.28918    -76.23826    -12.129937  ...  -35.51307     37.142223
   -47.257412 ]
 [ -25.834991    -7.7308607 -122.1515    ...  151.69942     83.41345
    55.42269  ]
 ...
 [  87.003365   -71.18344    112.54768   ...   92.26817    -81.561134
    -9.513627 ]
 [  48.042866    44.832867   -62.70945   ...  -72.722755    74.876144
  -131.32452  ]
 [-177.0037    -134.42023    -97.76626   ...  120.586624   190.59517
   -12.149335 ]]

reference:
 [[ -55.741196  -13.114624   27.261505 ...   30.763084  168.56616
    60.865753]
 [ 151.28903   -76.23833   -12.12993  ...  -35.513046   37.142227
   -47.257492]
 [ -25.835      -7.730755 -122.15131  ...  151.69925    83.413605
    55.42279 ]
 ...
 [  87.00337   -71.1834    112.54762  ...   92.268234  -81.560936
    -9.51362 ]
 [  48.042854   44.832924  -62.709427 ...  -72.72273    74.87613
  -13

Verify kernel results:

In [24]:
assert numpy.allclose(mojo_result, torch_result, rtol=0, atol=0.005)
for row in range(torch_result.shape[0]):
    if numpy.allclose(torch_result[row], mojo_result[row], rtol=0, atol=0.005): continue
    print('mismatch row:', row)

The tiled kernels can be visualized using the [matmul_visualization.mojo](./matmul_visualization.mojo) and `matmul_visualization_gui/*` scripts.

Here is an example screenshot:
![](./block_tiled_matrix_multiplication.png)

Run a complete benchmark for different algorithms:

In [25]:
import time
for algo in ["naive", "coalescing", "tiled", "tiled_register", "block_tiling"]:
    graph =  build_graph(session, algo)
    record = dict(torch=0, mojo=0)
    sampels = 5
    for _ in range(sampels):
        torch_A = torch.randn(M, K).to('cuda:0')
        torch_B = torch.randn(K, N).to('cuda:0')
        A = Tensor.from_numpy(torch_A.cpu().numpy()).to(device)
        B = Tensor.from_numpy(torch_B.cpu().numpy()).to(device)
        # torch
        torch.cuda.synchronize()
        begin = time.perf_counter()
        torch_result = torch_A @ torch_B
        torch.cuda.synchronize()
        record['torch'] += (time.perf_counter() - begin) / sampels
        # mojo
        torch.cuda.synchronize()
        begin = time.perf_counter()
        mojo_result = graph.execute(A, B)
        torch.cuda.synchronize()
        record['mojo'] += (time.perf_counter() - begin) / sampels
        assert numpy.allclose(mojo_result[0].to_numpy(), torch_result.cpu().numpy(), rtol=0, atol=0.005)
    print(algo, record)

building cuda graph for naive
loading cuda graph...
naive {'torch': 0.013316984591074287, 'mojo': 0.5164588708197698}
building cuda graph for coalescing
loading cuda graph...
coalescing {'torch': 0.013220496778376402, 'mojo': 0.1319526622071862}
building cuda graph for tiled
loading cuda graph...
tiled {'torch': 0.013328509824350476, 'mojo': 0.11017754999920724}
building cuda graph for tiled_register
loading cuda graph...
tiled_register {'torch': 0.013447657995857298, 'mojo': 0.404940522601828}
building cuda graph for block_tiling
loading cuda graph...
block_tiling {'torch': 0.01321662578266114, 'mojo': 1.1707371453754603}


## Reference
[1] https://github.com/modular/modular/blob/main/examples/custom_ops/kernels/matrix_multiplication.mojo

[2] https://docs.modular.com/max/tutorials/custom-ops-matmul